<a href="https://colab.research.google.com/github/ShlokShar/CourseConnect/blob/main/CourseConnectLang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
! pip install -U accelerate
! pip install -U transformers
import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, load_dataset, DatasetDict
from tqdm.auto import tqdm
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, get_scheduler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
keys = {
    0:'Parallelism',
    1:'Understatement',
    2:'Antithesis',
    3:'Epithet',
    4:'Aphorism',
    5:'Hyperbole',
    6:'Pathos',
    7:'Ethos',
    8:'Periodic_Sentence',
    9:'Anaphora',
    10:'Syllogism',
    11:'Euphemism',
    12:'Cumulative_Sentence',
    13:'Paradox',
    14:'Logos',
    15:'Apostrophe',
    16:'Allusion',
    17:'Balanced_Sentence',
    18:'Epigram'
}

dontTrain = ['Metaphor','Irony','Induction','Deduction','Connotation','Inversion','Simile','Tone','Alliteraiton','Interrupted_Sentence']

df = pd.DataFrame()

for term in keys:

    df2 = pd.read_csv(f'{keys[term]}_data.csv')
    df = df.append(df2, ignore_index=True)


df['term'] = df['term'].map({v: k for k, v in keys.items()})
df = df[['term','sentence']]
df = df.dropna()

print(df)
df.to_csv('allLangData.csv')

FileNotFoundError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
dataset = load_dataset("yelp_review_full")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
local_csv = load_dataset('csv',split='train',data_files='allLangData.csv')
local_csv = local_csv.train_test_split(test_size=0.20146058927)
filteredDataset = local_csv.remove_columns(["Unnamed: 0"])
filteredDataset = filteredDataset.rename_column("sentence", "text")
filteredDataset = filteredDataset.rename_column("term", "label")
filteredDataset = filteredDataset.shuffle(seed=42)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(filteredDataset['train']['text'][0:100])
print(dataset['train']['text'][0:100])

['I have some minor concerns with this project." (having major issues with a project)', 'Winning the lottery would be a nice surprise.', 'The weather isn\'t great today." - as a hurricane approaches.', "One man's trash is another man's treasure.", 'In order to lead, one must also learn to follow.', 'She was confident, persuasive, and resilient.', 'Love is an irresistible desire to be irresistibly desired." - Robert Frost', 'I only have a million things to do before the party tonight.', 'The land was dark, the sky was gray, the future was uncertain.', 'Success is not final, failure is not fatal: It is the courage to continue that counts.', 'He worked tirelessly to build his career, to build his wealth, and to build his reputation.', 'That was less than ideal." (terrible outcome)', 'We must not confuse dissent with disloyalty, and criticism with subversion.', 'Life is a marathon, not a sprint.', 'Evil men fear authority; good men cherish it." - Thomas Paine', 'It was the best of times, i

NameError: ignored

In [ ]:
tokenized_datasets = filteredDataset.map(tokenize_function, batched=True)

NameError: ignored

In [ ]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12639
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3189
    })
})


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=30)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=4,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
print(tokenized_datasets['train']['input_ids'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.568100,0.544093,0.845371
2,0.356700,0.471093,0.873485
3,0.222800,0.530622,0.875424
4,0.162000,0.515461,0.881241


TrainOutput(global_step=4088, training_loss=0.4143142464576169, metrics={'train_runtime': 3316.0296, 'train_samples_per_second': 9.861, 'train_steps_per_second': 1.233, 'total_flos': 8605894494412800.0, 'train_loss': 0.4143142464576169, 'epoch': 4.0})

In [ ]:
output_dir = "/model"
trainer.save_model(output_dir)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('/Model', num_labels=30)

In [ ]:
keys = {
    0:'Parallelism',
    1:'Understatement',
    2:'Antithesis',
    3:'Epithet',
    4:'Aphorism',
    5:'Hyperbole',
    6:'Pathos',
    7:'Ethos',
    8:'Periodic_Sentence',
    9:'Anaphora',
    10:'Syllogism',
    11:'Euphemism',
    12:'Cumulative_Sentence',
    13:'Paradox',
    14:'Logos',
    15:'Apostrophe',
    16:'Allusion',
    17:'Balanced_Sentence',
    18:'Epigram'
}

In [ ]:
sentence1 = "We shall fight in France, we shall fight on the seas and oceans, we shall fight with growing confidence and growing strength in the air, we shall defend our island whatever the cost may be"

encoding = tokenizer(sentence1, return_tensors='pt', padding=True, truncation=True)

output = model(**encoding)

logits = output.logits

top3_indices = torch.topk(logits, k=3, dim=1).indices[0].tolist()

# Get the corresponding class labels and probabilities
top3_classes = [keys[i] for i in top3_indices]
top3_probabilities = [probabilities[i] for i in top3_indices]

# Print the top 3 predicted classes and their probabilities
for i in range(3):
    print(f"Prediction {i + 1}: {top3_classes[i]}, Probability: {top3_probabilities[i]}")



Prediction 1: Anaphora, Probability: 0.8043568730354309
Prediction 2: Parallelism, Probability: 0.08866912126541138
Prediction 3: Antithesis, Probability: 0.05775376036763191


In [ ]:
from google.colab import files
files.download("/model/pytorch_model.bin")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>